In [100]:
import numpy as np
import pandas as pd
!apt install libgraphviz-dev
!pip install pygraphviz

def safelog2(x):
    if x == 0:
        return(0)
    else:
        return(np.log2(x))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,120 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-common all 2.24.32-1ubuntu1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-0 amd64 2.24.32-1ubuntu1 [1,769 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgail18 amd64 2.24.32-1ubuntu1 [14.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgail-common amd64 2.24.32

In [101]:
data_set = np.genfromtxt("hw05_data_set.csv", delimiter = ",")

X = data_set[:,0]
y = data_set[:,1].astype(int)

K = np.max(y)
N = X.shape[0]
D = 1
P = 25

train = np.array(data_set[1:150])
x_train = np.array([p[0] for p in train])
y_train = np.array([p[1] for p in train])

test = np.array(data_set[151:272])
x_test = np.array([p[0] for p in test])
y_test = np.array([p[1] for p in test])

N_train = len(y_train)
N_test = len(y_test)

In [102]:
def tree(x_train,y_train,P):
    node_indices = {}
    is_terminal = {}
    need_split = {}
    node_means = {}
    node_splits = {}

    node_indices[1] = np.array(range(len(x_train)))
    is_terminal[1] = False
    need_split[1] = True

In [103]:
while True:

    split_nodes = [key for key, value in need_split.items() if value == True]

    if len(split_nodes) == 0:
        break

    for split_node in split_nodes:
        data_indices = node_indices[split_node]
        need_split[split_node] = False
        node_frequencies[split_node] = [np.sum(y_train[data_indices] == c + 1) for c in range(K)]

        if x_train[data_indices].size <=P:
            is_terminal[split_node] = True
            node_means[split_node] = node_mean

        else:
            is_terminal[split_node] = False

            best_scores = np.repeat(0.0, 1)
            best_splits = np.repeat(0.0, 1)
            unique_values = np.sort(np.unique(x_train[data_indices]))
            split_positions = (unique_values[1:len(unique_values)] + unique_values[0:(len(unique_values) - 1)]) / 2
            split_scores = np.repeat(0.0, len(split_positions))
            for s in range(len(split_positions)):
                left_indices = data_indices[x_train[data_indices, 1] > split_positions[s]]
                right_indices = data_indices[x_train[data_indices, 1] <= split_positions[s]]
                split_scores[s] = -len(left_indices) / len(data_indices) * np.sum([np.mean(y_train[left_indices] == c + 1) * safelog2(np.mean(y_train[left_indices] == c + 1)) for c in range(K)]) - len(right_indices) / len(data_indices) * np.sum([np.mean(y_train[right_indices] == c + 1) * safelog2(np.mean(y_train[right_indices] == c + 1)) for c in range(K)])
            best_scores[1] = np.min(split_scores)
            best_splits[1] = split_positions[np.argmin(split_scores)]

            split_d = np.argmin(best_scores)

            node_features[split_node] = split_d
            node_splits[split_node] = best_splits[split_d]

            left_indices = data_indices[x_train[data_indices, split_d] > best_splits[split_d]]
            node_indices[2 * split_node] = left_indices
            is_terminal[2 * split_node] = False
            need_split[2 * split_node] = True
      
            right_indices = data_indices[X_train[data_indices, split_d] <= best_splits[split_d]]
            node_indices[2 * split_node + 1] = right_indices
            is_terminal[2 * split_node + 1] = False
            need_split[2 * split_node + 1] = True

In [104]:
def rmse(y, y_hat, P):
    r = np.sqrt(np.sum((y - y_hat) ** 2 / N_test))
    print("RMSE is", np.round(r, 4), "when P is", P)
    return r

In [105]:
def predict(x, node_splits, node_means, is_terminal):
    index = 1
    while 1:
        if is_terminal[index] == True:
            return node_means[index]
        if x > node_splits[index]:
            index = index*2 + 1
        else:
            index = index*2

In [107]:
def draw_tree():
    import pygraphviz as pgv

    class_colors = ["darkred", "darkgreen", "darkblue"]

    T = pgv.AGraph(directed = True, strict = True, rankdir = "UB")

    for key, value in is_terminal.items():
        if is_terminal[key] == False:
            T.add_node(key, shape = "ellipse", color = "black", 
                       label = "Node: {:d}\nx{:d} > {:.2f}\n[{:d}-{:d}-{:d}]".format(key,
                                                                                     node_features[key] + 1,
                                                                                     node_splits[key],
                                                                                     node_frequencies[key][0],
                                                                                     node_frequencies[key][1],
                                                                                     node_frequencies[key][2]))
        else:
            T.add_node(key, shape = "rectangle", 
                       color = class_colors[np.argmax(node_frequencies[key])],
                       fontcolor = class_colors[np.argmax(node_frequencies[key])],
                       label = "Node: {:d}\nMajority: {:d}\n[{:d}-{:d}-{:d}]".format(key,
                                                                                     np.argmax(node_frequencies[key]) + 1,
                                                                                     node_frequencies[key][0],
                                                                                     node_frequencies[key][1],
                                                                                     node_frequencies[key][2]))

    for key, value in is_terminal.items():
        if key > 1:
            if key % 2 == 0:
                T.add_edge(key // 2, key, label = "True", color = "black")
            else:
                T.add_edge(key // 2, key, label = "False", color = "black")

    T.layout("dot")

draw_tree()
import matplotlib.pyplot as plt
plt.figure(figsize = (8, 10))
plt.axis("off")
plt.show()

KeyError: ignored

In [108]:
terminal_nodes = [key for key, value in is_terminal.items() if value == True]
for terminal_node in terminal_nodes:
    index = terminal_node
    rules = np.array([])
    while index > 1:
        parent = np.floor(index / 2)
        if index % 2 == 0:
            rules = np.append(rules, "x{:d} > {:.2f}".format(node_features[parent] + 1, node_splits[parent]))
        else:
            rules = np.append(rules, "x{:d} <= {:.2f}".format(node_features[parent] + 1, node_splits[parent]))
        index = parent
    rules = np.flip(rules)
    print("{} => {}".format(rules, node_frequencies[terminal_node]))